In [4]:
import numpy as np
import pandas as pd
import xgboost as xgb
import shap
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_validate, cross_val_score, cross_val_predict
from sklearn.metrics import matthews_corrcoef, confusion_matrix, roc_auc_score, classification_report
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.combine import SMOTETomek
from imblearn.pipeline import Pipeline
from sklearn.metrics import recall_score, accuracy_score
from sklearn.impute import KNNImputer, SimpleImputer

In [5]:
random_state = 7
cv_method = StratifiedKFold(n_splits=5)

# Training of tsfresh-selected Training Max 1 years and eGFR 4 times

In [6]:
df = pd.read_csv('/home/jupyter-dchristiadi85/PhD Project 1/data/train_selected_1_4.gz')
df.shape

(5603, 23639)

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5603 entries, 0 to 5602
Columns: 23639 entries, id to cat8
dtypes: float64(23617), int64(2), object(20)
memory usage: 1010.5+ MB


In [8]:
df['aki_1'].fillna(value=0, inplace=True)
df['aki_2'].fillna(value=0, inplace=True)
df['aki_3'].fillna(value=0, inplace=True)

In [9]:
df['gn'].value_counts()

no                 5181
igan                172
anca                 76
lupus nephritis      39
membranous           33
unspecific           30
fsgs                 28
mcd                  21
mp/mcgn              16
fgn/itg               5
pign                  2
Name: gn, dtype: int64

In [10]:
df.loc[df.gn == 'pign', 'gn'] = 'unspecific'
df.loc[df.gn == 'fgn/itg', 'gn'] = 'unspecific'

In [11]:
percent_missing = df.isnull().sum()*100 / len(df)
missing_df = pd.DataFrame({'column_name': df.columns, 'percent_miss': percent_missing})

In [12]:
missing_df.sort_values(by='percent_miss', ascending=False)

,column_name,percent_miss
"mch__fft_coefficient__attr_""""""""""""""""real""""""""""""""""__coeff_71","mch__fft_coefficient__attr_""""""""""""""""real""""""""""""""...",100.0
"platelet__fft_coefficient__attr_""""""""""""""""abs""""""""""""""""__coeff_75","platelet__fft_coefficient__attr_""""""""""""""""abs""""""...",100.0
"platelet__fft_coefficient__attr_""""""""""""""""angle""""""""""""""""__coeff_91","platelet__fft_coefficient__attr_""""""""""""""""angle""...",100.0
"platelet__fft_coefficient__attr_""""""""""""""""angle""""""""""""""""__coeff_90","platelet__fft_coefficient__attr_""""""""""""""""angle""...",100.0
"platelet__fft_coefficient__attr_""""""""""""""""angle""""""""""""""""__coeff_89","platelet__fft_coefficient__attr_""""""""""""""""angle""...",100.0
...,...,...
htn,htn,0.0
aki_3,aki_3,0.0
aki_2,aki_2,0.0
aki_1,aki_1,0.0


### Number of patients reaching ESKD within training period 

In [13]:
df['cat1'].value_counts()

non_eskd    4409
eskd        1194
Name: cat1, dtype: int64

In [14]:
df.set_index('id', inplace=True)

In [15]:
def to_category(df):
    cols = df.select_dtypes(include='object').columns
    for col in cols:
        df[col] = df[col].astype('category')
    return df

def drop_missing(df):
    threshold = len(df)*0.6
    df.dropna(axis=1, thresh=threshold, inplace=True)
    return df

def copy_df(df):
    return df.copy()

In [16]:
df_cleaned = (df.pipe(copy_df).pipe(drop_missing).pipe(to_category))
df_cleaned.shape

(5603, 5361)

In [17]:
df_rm_intrain = df_cleaned.query("cat1 == 'non_eskd'")
df_rm_intrain.shape

(4409, 5361)

In [18]:
df_rm_intrain.to_csv('../data/cleaned60_selected_1_4.gz', index=True, compression='gzip')

In [19]:
df_rm_intrain.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4409 entries, 3916 to 23864
Columns: 5361 entries, mchc__variance_larger_than_standard_deviation to cat8
dtypes: category(20), float64(5340), int64(1)
memory usage: 179.8 MB


In [112]:
df_rm_intrain = (df_rm_intrain.pipe(copy_df).pipe(to_category))
df_rm_intrain.shape

(4409, 8985)

### Number of patients reaching ESKD in at 1y post-training 

In [20]:
df_rm_intrain['cat2'].value_counts()

non_eskd    4282
eskd         127
Name: cat2, dtype: int64

### Number of patients reaching ESKD in 2y post-training

In [21]:
df_rm_intrain['cat3'].value_counts()

non_eskd    4174
eskd         235
Name: cat3, dtype: int64

### Number of patients reaching ESKD in 5y post-training

In [22]:
df_rm_intrain['cat6'].value_counts()

non_eskd    3918
eskd         491
Name: cat6, dtype: int64

In [23]:
percent_missing = df_rm_intrain.isnull().sum()*100 / len(df_rm_intrain)
missing_df_rm_intrain = pd.DataFrame({'column_name': df_rm_intrain.columns, 'percent_miss': percent_missing})

In [24]:
missing_df_rm_intrain.sort_values(by='percent_miss', ascending=False)

,column_name,percent_miss
caphos_product__autocorrelation__lag_0,caphos_product__autocorrelation__lag_0,47.153550
caphos_product__autocorrelation__lag_1,caphos_product__autocorrelation__lag_1,47.153550
caphos_product__fourier_entropy__bins_100,caphos_product__fourier_entropy__bins_100,47.130869
caphos_product__fourier_entropy__bins_10,caphos_product__fourier_entropy__bins_10,47.130869
caphos_product__fourier_entropy__bins_5,caphos_product__fourier_entropy__bins_5,47.130869
...,...,...
dkd,dkd,0.000000
htn,htn,0.000000
aki_3,aki_3,0.000000
aki_2,aki_2,0.000000


## Features and Target Input for Training 

In [25]:
dropped_cols = list(df_rm_intrain.columns[-18:])
dropped_cols.remove('age.init')
dropped_cols

['egfr.y',
 'cat0.5',
 'cat1',
 'cat1.5',
 'cat2',
 'cat2.5',
 'cat3',
 'cat3.5',
 'cat4',
 'cat4.5',
 'cat5',
 'cat5.5',
 'cat6',
 'cat6.5',
 'cat7',
 'cat7.5',
 'cat8']

In [26]:
X = df_rm_intrain.drop(dropped_cols,axis=1).copy()
y = df_rm_intrain['cat6']
X.shape, y.shape

((4409, 5344), (4409,))

In [27]:
y_mapped = y.map({'non_eskd':'no', 'eskd':'yes'})
y_mapped

id
3916     no
3918     no
3921     no
3924     no
3930     no
         ..
23760    no
23771    no
23780    no
23807    no
23864    no
Name: cat6, Length: 4409, dtype: category
Categories (2, object): ['yes', 'no']

In [28]:
X_train, X_test, y_train, y_test = train_test_split(X, y_mapped, test_size=0.3, stratify=y_mapped, random_state=random_state)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((3086, 5344), (1323, 5344), (3086,), (1323,))

In [29]:
lab_encode = LabelEncoder()
y_train = lab_encode.fit_transform(y_train)
y_test = lab_encode.transform(y_test)
y_train.shape, y_test.shape

((3086,), (1323,))

In [30]:
unique_values, counts = np.unique(y_train, return_counts=True)
ratio = dict(zip(unique_values, counts))
ratio

{0: 2742, 1: 344}

In [31]:
imbalance_ratio = ratio[0] / ratio[1]
imbalance_ratio

7.97093023255814

In [32]:
category_cols = list(X_train.select_dtypes(include='category').columns)
numeric_cols = list(X_train.select_dtypes(include=['number']).columns)
one_hot = OneHotEncoder(handle_unknown='ignore')
num_imputer = KNNImputer(n_neighbors=5)
len(category_cols), len(numeric_cols)

(4, 5340)

In [35]:
preprocess = ColumnTransformer([('numerical', num_imputer, numeric_cols), ('cat_encoder', one_hot, category_cols)], remainder='passthrough')
X_train_processed = preprocess.fit_transform(X_train)
X_train_processed.shape

(3086, 5355)

In [36]:
X_test_processed = preprocess.transform(X_test)
X_test_processed.shape

(1323, 5355)

In [37]:
xgb_class = xgb.XGBClassifier(n_jobs=-1, random_state = random_state, n_estimators=1000, use_label_encoder=False, eval_metric='logloss', scale_pos_weight=imbalance_ratio)
sampler = SMOTETomek(sampling_strategy=0.5, random_state=random_state)
steps = [('resampling', sampler), ('model', xgb_class)]
pipeline = Pipeline(steps=steps)

In [38]:
pipeline.fit(X_train_processed, y_train)

Pipeline(steps=[('resampling',
                 SMOTETomek(random_state=7, sampling_strategy=0.5)),
                ('model',
                 XGBClassifier(base_score=0.5, booster='gbtree',
                               colsample_bylevel=1, colsample_bynode=1,
                               colsample_bytree=1, eval_metric='logloss',
                               gamma=0, gpu_id=-1, importance_type='gain',
                               interaction_constraints='',
                               learning_rate=0.300000012, max_delta_step=0,
                               max_depth=6, min_child_weight=1, missing=nan,
                               monotone_constraints='()', n_estimators=1000,
                               n_jobs=-1, num_parallel_tree=1, random_state=7,
                               reg_alpha=0, reg_lambda=1,
                               scale_pos_weight=7.97093023255814, subsample=1,
                               tree_method='exact', use_label_encoder=False,
      

In [39]:
y_pred = pipeline.predict(X_test_processed)

In [40]:
confusion_matrix(y_true=y_test, y_pred=y_pred)

array([[1116,   60],
       [  79,   68]])

In [41]:
roc_auc_score(y_test, y_pred)

0.7057823129251701

In [42]:
print(classification_report(y_true=y_test, y_pred=y_pred))

              precision    recall  f1-score   support

           0       0.93      0.95      0.94      1176
           1       0.53      0.46      0.49       147

    accuracy                           0.89      1323
   macro avg       0.73      0.71      0.72      1323
weighted avg       0.89      0.89      0.89      1323



In [43]:
print(matthews_corrcoef(y_test, y_pred))

0.43753362094365605


In [44]:
importance = pipeline.named_steps['model'].feature_importances_
importance.shape, type(importance)

((5355,), numpy.ndarray)

In [46]:
category_label = list(preprocess.named_transformers_['cat_encoder'].get_feature_names(category_cols))
feature_label = numeric_cols + category_label
len(feature_label)

5355

In [47]:
pd.set_option('display.max_rows', 400)
eval_df = pd.DataFrame({'label': feature_label, 'importance_value': importance})
eval_df.sort_values(by='importance_value', ascending=False).head(100)

,label,importance_value
3572,creatinine__quantile__q_0.9,0.126166
3515,creatinine__maximum,0.019196
3571,creatinine__quantile__q_0.8,0.015633
5108,chloride__standard_deviation,0.014214
1278,"potassium__change_quantiles__f_agg_""""""""""""""""mea...",0.010256
3761,egfr__c3__lag_1,0.009101
2671,"albumin__change_quantiles__f_agg_""""""""""""""""mean""...",0.008887
2028,urea__energy_ratio_by_chunks__num_segments_10_...,0.008486
624,neutrophils__energy_ratio_by_chunks__num_segme...,0.008415
395,monocytes__range_count__max_1__min_-1,0.008190
